In [1]:
# 데이터 전처리 및 한글 데이터 형대소 분석 패키지
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import konlpy

In [2]:
# Matplotlib 그래프를 위한 설정
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 한국어 띄어쓰기를 교정해주는 라이브러리
from pykospacing import Spacing
# 한글 형태소 분석기 okt
from konlpy.tag import Okt
okt = Okt()

def correct_spacing(text):
    return ' '.join(okt.morphs(text))

# 예제 문장
text = "오늘날씨가정말좋네요"
corrected_text = correct_spacing(text)
print("Okt : ",corrected_text)

spacing = Spacing()
text = "오늘날씨가정말좋네요"
corrected_text = spacing(text)
print("Spacing : ",corrected_text)

# Okt 형태소 분석기 테스트
print(okt.morphs('아름다운 날씨가 너무 좋아요. 내일은 더울 것 같아요.'))
print(okt.pos('아름다운 날씨가 너무 좋아요. 내일은 더울 것 같아요. 그러나 토익 공부한다'))
# 명사, 형용사, 동사와 같이 특정 품사만 추출
print(okt.nouns('아름다운 날씨가 너무 좋아요. 내일은 더울 것 같아요.'))
# 명사, 형용사, 부사, 동사 만 남기는 함수
def extract_nouns_and_verbs(text):
    return [word for word, pos in okt.pos(text) if pos in ['Noun', 'Adjective', 'Verb', 'Adverb']]
print(extract_nouns_and_verbs('아름다운 날씨가 너무 좋다. 내일은 더울 것 같아요.'))


Okt :  오늘날 씨 가 정말 좋네요
Spacing :  오늘 날씨가 정말 좋네요
['아름다운', '날씨', '가', '너무', '좋아요', '.', '내일', '은', '더울', '것', '같아요', '.']
[('아름다운', 'Adjective'), ('날씨', 'Noun'), ('가', 'Josa'), ('너무', 'Adverb'), ('좋아요', 'Adjective'), ('.', 'Punctuation'), ('내일', 'Noun'), ('은', 'Josa'), ('더울', 'Adjective'), ('것', 'Noun'), ('같아요', 'Adjective'), ('.', 'Punctuation'), ('그러나', 'Conjunction'), ('토익', 'Noun'), ('공부', 'Noun'), ('한다', 'Verb')]
['날씨', '내일', '것']
['아름다운', '날씨', '너무', '좋다', '내일', '더울', '것', '같아요']


In [4]:
# 한글 댓글의 특수문자, 불용어 제거 및 공백, 줄바꿈 처리
def clean_comments(text):
    # 여러줄로 되있는 것을 한줄로 변환
    text = ' '.join(text.splitlines())
    # 시간 표현 제거
    text = re.sub(r'\b\d+:\d+\b', '', text)
    # 특수 문자 및 한글이 아닌 문자 제거, 영어와 숫자는 남김
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)  # 한글, 영어, 숫자, 공백을 제외한 모든 문자 제거
    text = re.sub(r'\s+', ' ', text)  # 공백과 줄바꿈을 하나의 공백으로 변환
    text = spacing(text) # 한글 뛰어쓰기 안된 문장 뛰어쓰기로 변환

    return text.strip()  # 앞뒤 공백 제거

# 댓글 데이터 예시
text1 = """
약사님 ~??지금 병원 처방 유산균, 과민성 대장 증후군 약과 갑상선 암 반절제 수술(작년) 후
갑자기 혈압이 높아져서 약을 복용 중이며 2년전? 약사님 영상에서도 봤고 약국에서 아로나민골드 프리미엄 추천해 주셔서 먹는데 셀레늄이나 아연 비타민 D 등이 추가적으로 필요할까요?
혹시 더 필요한 것들은 무엇이 있을까요? 혈압에 ..아직도 암환자같고 영양제 너무 어렵네요??30살입니다!
"""
example_comments = [
    "3:59 아이허브 보니 이제품에듀 실라짓 들어간듯한데 그럼 이것도 통관불가 아닐지요?",
    text1,
    "닥터스 베스트는 하루에 3알 먹어야 되는 종비인거죠?",
    "안녕하세요저는부산에사는60대여성입니다문의드릴께있어서요샤프란이라는건강식품은어떤효능이있..."
]

# 댓글 데이터 클리닝
cleaned_comments = [clean_comments(comment) for comment in example_comments]
print(cleaned_comments)
# cleaned_comments = [extract_nouns_and_verbs(comment) for comment in cleaned_comments]
# print(cleaned_comments)



['아이허브 보니 이 제품에듀 실라짓 들어간 듯한 데 그럼 이것도 통관 불가 아닐지요', '약사님 지금 병원 처방 유산균 과 민성 대장 증후군 약과 갑상선 암 반절제 수술 작년 후 갑자기 혈압이 높아져서 약을 복용 중이며 2년 전 약사님 영상에서도 봤고 약국에서 아로나민골드 프리미엄 추천해 주셔서 먹는데 셀레늄이나 아연 비타민 D 등이 추가적으로 필요할까요 혹시 더 필요한 것들은 무엇이 있을까요 혈압에 아직도 암 환자 같고 영양제 너무 어렵네요 30살 입니다', '닥터스 베스트는 하루에 3 알 먹어야 되는 종비인 거죠', '안녕하세 요저는 부산에 사는 60대 여성입니다문의 드릴께 있어서 요샤 프란이라는 건강식품은 어떤 효능이 있']


In [5]:
# 댓글 데이터를 전저리하기 위해 데이터 파일 읽기
df = pd.read_csv('./youtube_channel_comments_data_20240601_035208.csv')
print(df.head())

                  channelId    channelTitle    channelPublishedAt  \
0  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
1  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
2  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
3  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
4  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   

   subscriberCount      videoId  \
0          1780000  RAO61ojyDDc   
1          1780000  RAO61ojyDDc   
2          1780000  RAO61ojyDDc   
3          1780000  RAO61ojyDDc   
4          1780000  RAO61ojyDDc   

                                          videoTitle  \
0  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
1  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
2  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
3  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
4  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   

              videoAuthorId      vide

In [6]:
# 전체 데이터의 각 열의 데이터 타입을 확인
print(df.dtypes)

channelId             object
channelTitle          object
channelPublishedAt    object
subscriberCount        int64
videoId               object
videoTitle            object
videoAuthorId         object
videoPublishedAt      object
viewCount              int64
likeCount              int64
dislikeCount           int64
commentId             object
commentAuthor         object
authorId              object
commentText           object
commentLikeCount       int64
commentPublishedAt    object
parentCommentId       object
dtype: object


In [7]:
# 데이터 프레임의 commentText 열의 데이터 타입을 변환
df['commentText'] = df['commentText'].astype(str)
# commentText 열의 데이를 전처리하고 새로운 열을 추가하기 위해 함수 적용
df['cleaned_comment'] = df['commentText'].apply(clean_comments)
# df['cleaned_comment'] = df['cleaned_comment'].apply(extract_nouns_and_verbs)

print(df.head())# 5개의 데이터 보기


                  channelId    channelTitle    channelPublishedAt  \
0  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
1  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
2  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
3  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   
4  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기  2017-03-02T02:28:38Z   

   subscriberCount      videoId  \
0          1780000  RAO61ojyDDc   
1          1780000  RAO61ojyDDc   
2          1780000  RAO61ojyDDc   
3          1780000  RAO61ojyDDc   
4          1780000  RAO61ojyDDc   

                                          videoTitle  \
0  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
1  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
2  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
3  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
4  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   

              videoAuthorId      vide

In [8]:
print(df.dtypes)

channelId             object
channelTitle          object
channelPublishedAt    object
subscriberCount        int64
videoId               object
videoTitle            object
videoAuthorId         object
videoPublishedAt      object
viewCount              int64
likeCount              int64
dislikeCount           int64
commentId             object
commentAuthor         object
authorId              object
commentText           object
commentLikeCount       int64
commentPublishedAt    object
parentCommentId       object
cleaned_comment       object
dtype: object


In [9]:
# 데이터 분석을 위해 날짜 데이터 열(channelPublishedAt, videoPublishedAt, commentPublishedAt)을 
# 오브젝트타입에서  datetime으로 변경
# 날짜 데이터 열의 데이터 타입을 datetime으로 변경
# df['channelPublishedAt'] = pd.to_datetime(df['channelPublishedAt'])
# df['videoPublishedAt'] = pd.to_datetime(df['videoPublishedAt'])
# df['commentPublishedAt'] = pd.to_datetime(df['commentPublishedAt'])
df['channelPublishedAt'] = pd.to_datetime(df['channelPublishedAt'], format='mixed')
df['videoPublishedAt'] = pd.to_datetime(df['videoPublishedAt'], format='mixed')
df['commentPublishedAt'] = pd.to_datetime(df['commentPublishedAt'], format='mixed')

In [10]:
# 공백이나 N/A 데이터 확인
print(df.isnull().sum())


channelId                 0
channelTitle              0
channelPublishedAt        0
subscriberCount           0
videoId                   0
videoTitle                0
videoAuthorId             0
videoPublishedAt          0
viewCount                 0
likeCount                 0
dislikeCount              0
commentId                 0
commentAuthor             1
authorId                  0
commentText               0
commentLikeCount          0
commentPublishedAt        0
parentCommentId       13923
cleaned_comment           0
dtype: int64


In [11]:
# commentText 를 cleaned_comment 로 바꾸고 cleaned_comment는 삭제
df['commentText'] = df['cleaned_comment']
df = df.drop(columns=['cleaned_comment'])
print(df.head())



                  channelId    channelTitle        channelPublishedAt  \
0  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
1  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
2  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
3  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   
4  UCMFk5S7g5DY-CZNVh_Kyz_A  약사가 들려주는 약 이야기 2017-03-02 02:28:38+00:00   

   subscriberCount      videoId  \
0          1780000  RAO61ojyDDc   
1          1780000  RAO61ojyDDc   
2          1780000  RAO61ojyDDc   
3          1780000  RAO61ojyDDc   
4          1780000  RAO61ojyDDc   

                                          videoTitle  \
0  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
1  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
2  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
3  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   
4  혈당을 뚝 떨어뜨리고 당뇨병을 예방 하는 가성비 좋은 베르베린부터 최고 품질의 베르...   

             

In [12]:
# 변환된 데이터를 csv 파일로 저장
df.to_csv('./youtube_channel_comments_cleaneddata.csv', index=False)